### Import libraries

In [1]:
!pip install --upgrade tensorflow-io

In [16]:
import pandas as pd
import os
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model, Sequential, Sequential
from tensorflow.keras.layers import Input, LSTM, Embedding,Bidirectional, Dense,Activation,TimeDistributed, RepeatVector, GRU, Attention

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [17]:
df=pd.read_csv("/kaggle/input/medium-articles-dataset/medium_data.csv")
df

,id,url,title,subtitle,image,claps,responses,reading_time,publication,date
0,1,https://towardsdatascience.com/a-beginners-gui...,A Beginner’s Guide to Word Embedding with Gens...,NaN,1.png,850,8,8,Towards Data Science,2019-05-30
1,2,https://towardsdatascience.com/hands-on-graph-...,Hands-on Graph Neural Networks with PyTorch & ...,NaN,2.png,1100,11,9,Towards Data Science,2019-05-30
2,3,https://towardsdatascience.com/how-to-use-ggpl...,How to Use ggplot2 in Python,A Grammar of Graphics for Python,3.png,767,1,5,Towards Data Science,2019-05-30
3,4,https://towardsdatascience.com/databricks-how-...,Databricks: How to Save Files in CSV on Your L...,When I work on Python projects dealing…,4.jpeg,354,0,4,Towards Data Science,2019-05-30
4,5,https://towardsdatascience.com/a-step-by-step-...,A Step-by-Step Implementation of Gradient Desc...,One example of building neural…,5.jpeg,211,3,4,Towards Data Science,2019-05-30
...,...,...,...,...,...,...,...,...,...,...
6503,6504,https://medium.com/better-marketing/we-vs-i-ho...,“We” vs “I” — How Should You Talk About Yourse...,Basic copywriting choices with a big…,6504.jpg,661,6,6,Better Marketing,2019-12-05
6504,6505,https://medium.com/better-marketing/how-donald...,How Donald Trump Markets Himself,Lessons from who might be the most popular bra...,6505.jpeg,189,1,5,Better Marketing,2019-12-05
6505,6506,https://medium.com/better-marketing/content-an...,Content and Marketing Beyond Mass Consumption,How to acquire customers without wasting money...,6506.jpg,207,1,8,Better Marketing,2019-12-05
6506,6507,https://medium.com/better-marketing/5-question...,5 Questions All Copywriters Should Ask Clients...,Save time and effort by…,6507.jpg,253,2,5,Better Marketing,2019-12-05


### Text-preprocessing

In [4]:
import re
from string import punctuation,digits
replacements = {"they're": "they are","it's": "it is","that's": "that is","we're": "we are","you're":"you are","i'm":"i am","don't":"do not","what's":"what is"}
def preprocess(text): 
    text = re.compile(r'https?://\S+|www\.\S+').sub('', text)#links removal
    text = text.lower() #lowercase removal
    text = re.sub(r'\d','',text) #digit removal
    text = re.compile(r'\s+').sub(' ',text) #white space removal
    for old, new in replacements.items():
        text = text.replace(old, new)
    text=re.compile(r"[\u0900-\u097F]+").sub('',text)#remove hindi words
    text=re.compile(r'[^\w\s]').sub('',text)#remove all punctuation except  . , ? 
#     text=re.compile(r'[^a-zA-Z\s]').sub(r'', text)#unnecessary words Removal 
    return text

In [5]:
df['title']=df['title'].apply(preprocess)

### tokenize

In [6]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(df['title'])
sequence=tokenizer.texts_to_sequences(df['title'])

In [7]:
#word-dectionary
word_index=tokenizer.word_index
total_words=len(word_index)+1

### N-gram

In [8]:
max_length=max(len(line) for line in sequence)
print(max_length)

23


In [9]:
input_sequences = [lines[:i+1] for lines in sequence for i in range(1, len(lines))]
input_sequences = pad_sequences(input_sequences, maxlen=max_length, padding='pre')

In [21]:
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)
print(xs.shape)
print(labels.shape)
print(ys.shape)

(44104, 22)
(44104,)
(44104, 8433)


# GRU for word prediction

In [11]:
# Define the model architecture
embedding_dim=100
model = Sequential()
model.add(Embedding(total_words, embedding_dim, input_length=max_length-1))
model.add(GRU(units=64, return_sequences=True))
model.add(GRU(units=64))
model.add(Dense(total_words, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(xs, ys, epochs=200, batch_size=80)
print(model)

Epoch 1/200
552/552 [==============================] - 30s 45ms/step - loss: 7.4460 - accuracy: 0.0439
Epoch 2/200
552/552 [==============================] - 7s 13ms/step - loss: 6.8946 - accuracy: 0.0610
Epoch 3/200
552/552 [==============================] - 7s 13ms/step - loss: 6.5385 - accuracy: 0.0918
Epoch 4/200
552/552 [==============================] - 7s 13ms/step - loss: 6.1970 - accuracy: 0.1158
Epoch 5/200
552/552 [==============================] - 6s 12ms/step - loss: 5.9140 - accuracy: 0.1340
Epoch 6/200
552/552 [==============================] - 6s 11ms/step - loss: 5.6716 - accuracy: 0.1485
Epoch 7/200
552/552 [==============================] - 6s 11ms/step - loss: 5.4560 - accuracy: 0.1607
Epoch 8/200
552/552 [==============================] - 6s 11ms/step - loss: 5.2565 - accuracy: 0.1727
Epoch 9/200
552/552 [==============================] - 6s 11ms/step - loss: 5.0732 - accuracy: 0.1832
Epoch 10/200
552/552 [==============================] - 6s 10ms/step - loss: 4.90

In [15]:
text = "data science is"
next_words = 7

# Generate text using the trained model
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([text])[0]
    token_list = pad_sequences([token_list], maxlen=max_length-1, padding='pre')
    predictions = model.predict(token_list)
    predicted_word_index = np.argmax(predictions, axis=-1)
    predicted_word = tokenizer.index_word[predicted_word_index[0]]
    text += " " + predicted_word

print(text)

1/1 [==============================] - 0s 22ms/step
data science is real work description me it will good
